# About this notebook

This notebook ensembles result of 5 starter notebook. Its blending based on public score is a really simple, while its result can match the performance of other complex blending attempts with all availbale public kernels before competition end.

<br/><br/>
**The five starter kernels are:**
1. Using English translation and Bert-base: https://www.kaggle.com/miklgr500/jigsaw-tpu-bert-with-huggingface-and-keras (.9158)
2. XLM-R with augmentation: https://www.kaggle.com/yeayates21/xlm-roberta-augmentation-ssl-0-9417-pub-lb (.9399)
3. Finetune XLM-R by layer: https://www.kaggle.com/riblidezso/train-from-mlm-finetuned-xlm-roberta-large (.9422)
4. Pytorch XLM-R with augmentation: https://www.kaggle.com/shonenkov/tpu-training-super-fast-xlmroberta (.9416)
5. validset adjusted plus simple blending: https://www.kaggle.com/shonenkov/tpu-training-super-fast-xlmroberta (.9459)

(The fifth blends the fourth with a early version of hamditarek's ensemble about .9430)


<br/><br/>
**Example for complex blending before competition end:**
1. https://www.kaggle.com/hamditarek/ensemble?scriptVersionId=37216149 (version-108 .9471)
2. https://www.kaggle.com/jazivxt/howling-with-wolf-on-l-genpresse (.9472)


*In contrast, these simple blending solution gives .9473 before and .9482 after adjusting mean of each language follwing advice of [4th solution](https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification/discussion/160980).*


# Main idea

* This notebook shows the baseline score can be achieved by emsemble public kernels before competition end.
* Because of the huge similarity in public kernels avaible before competition end, starter kernels plus simple blending can achieve decent result among all pure blending solutions. 
* The result of this notebook is used to blended with [my monolingual models kernel](https://www.kaggle.com/mint101/jmtc-20-lb-9508-mono-lingual-models) to achieve lb.9514

In [ ]:
import numpy as np 
import pandas as pd 
from scipy.special import softmax

import os

In [ ]:
in_path = '/kaggle/input/'
path = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'
files = ['tpu-inference-super-fast-xlmroberta/submission.csv',
         'tpu-training-super-fast-xlmroberta/submission.csv',
         'train-from-mlm-finetuned-xlm-roberta-large/submission.csv',
         'xlm-roberta-augmentation-ssl-0-9417-pub-lb/submission.csv',
         'jigsaw-tpu-bert-with-huggingface-and-keras/submission.csv']   
sumbs = np.array([pd.read_csv(in_path+f).toxic for f in files])

pb = np.array([0.9459,0.9416,0.9422,0.9399,0.9158])

# Simple blending schedule

In [ ]:
"""Weighted by sofmax of 1 over 1 minus pb score."""
weight = lambda x: softmax(1/(1-x))

# Blending and Adjusting

In [ ]:
w = weight(pb)
print(w)
sub = pd.read_csv(path+'sample_submission.csv')
sub['toxic'] = sumbs.T@w
sub.head()

**(The reault here obtains lb.9473)**

In [ ]:
test = pd.read_csv(path+'test.csv')
dic_ids = {k:v.id for k,v in test.groupby(["lang"])}
adj = {
    "fr":1.04,
    "es":1.06,
    "pt":.96,
    "it":.97,
    "tr":.98,
}

for l,v in adj.items():
    ids = dic_ids[l]
    sub.loc[ids,"toxic"] *= v

sub.head()

# Submit

In [ ]:
sub.to_csv('submission.csv', index=False)